In [1]:
import sys
print(sys.executable)

C:\Users\lavin\anaconda3\envs\PythonData\python.exe


In [2]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [20]:
# import libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import pymongo
from pymongo import MongoClient
import certifi
from mongo2_config import mongo_uri
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

import datetime
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
plt.style.use('fivethirtyeight')

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [6]:
# create a dataframe from the csv
path ="C:/UC_Berkeley\Analysis_Projects_Class_Folder/Module20_Final_Project\Data_News_Cryptocomp\Final_News_df(5year_2016_2021).csv"
Final_News_df = pd.read_csv(path)
Final_News_df

,Unnamed: 0,title,source,body,categories,source_info.name,Coin_Symbol,Rev_time,Dt_Rev_time
0,0,SuperNET Weekly No. 18,coremediasupernet,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11
1,1,SuperNET Weekly No. 18,coremediasupernet,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11
2,2,SuperNET Weekly No. 18,coremediasupernet,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11
3,3,SuperNET Weekly No. 18,coremediasupernet,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11
4,4,SuperNET Weekly No. 18,coremediasupernet,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,DOGE,2016-07-11 00:00:00,2016-07-11
...,...,...,...,...,...,...,...,...,...
344053,95831,"Fintechs are hungry for DeFi yields, says Comp...",theblock,"DeFi and traditional finance are converging, s...",ETH|Blockchain,TheBlock,DOGE,2021-07-06 19:05:16,2021-07-06
344054,95832,Why this is crucial to XRP’s trajectory over t...,ambcrypto,The global cryptocurrency market noted a bit o...,XRP|Market|Trading|Altcoin,AMB Crypto,DOGE,2021-07-06 19:00:37,2021-07-06
344055,95833,Ripple CTO slams regulators as Barclays blocks...,cryptoslate,Barclays UK customers have been blocked from t...,Exchange|XRP|Regulation|Fiat,CryptoSlate,DOGE,2021-07-06 19:00:21,2021-07-06
344056,95834,Crypto Head: The U.S. Is the “Most-Ready” Nati...,livebitcoinnews,According to a new report issued by cryptocurr...,BTC,Live Bitcoin News,DOGE,2021-07-06 19:00:11,2021-07-06


In [7]:
Final_News_df.drop(columns=['Unnamed: 0', 'source'],axis=1, inplace=True)
Final_News_df

,title,body,categories,source_info.name,Coin_Symbol,Rev_time,Dt_Rev_time
0,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11
1,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11
2,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11
3,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11
4,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,DOGE,2016-07-11 00:00:00,2016-07-11
...,...,...,...,...,...,...,...
344053,"Fintechs are hungry for DeFi yields, says Comp...","DeFi and traditional finance are converging, s...",ETH|Blockchain,TheBlock,DOGE,2021-07-06 19:05:16,2021-07-06
344054,Why this is crucial to XRP’s trajectory over t...,The global cryptocurrency market noted a bit o...,XRP|Market|Trading|Altcoin,AMB Crypto,DOGE,2021-07-06 19:00:37,2021-07-06
344055,Ripple CTO slams regulators as Barclays blocks...,Barclays UK customers have been blocked from t...,Exchange|XRP|Regulation|Fiat,CryptoSlate,DOGE,2021-07-06 19:00:21,2021-07-06
344056,Crypto Head: The U.S. Is the “Most-Ready” Nati...,According to a new report issued by cryptocurr...,BTC,Live Bitcoin News,DOGE,2021-07-06 19:00:11,2021-07-06


In [4]:
Final_News_df.shape

(344058, 9)

In [17]:
#combine the top news headlines
headlines = []

for row in range(0, len(Final_News_df.index)):
    headlines.append(' '.join( str(x) for x in Final_News_df.iloc[row, 0:2]))

In [18]:
#print a sample of combined headlines
headlines[0]

'SuperNET Weekly No. 18 CORE and CORE Media were birthed out of the community which formed around the SuperNET project launched by jl777 in 2014.\xa0Now that many aspects of the project are finally coming together in fantastic form and usable products are just around the bend, we couldn\'t be more pleased to help spread the word and keep our audience up to date on the latest happenings with this revolutionary project and its technology. Welcome to SuperNET Weekly!SuperNET Featured in CORE Magazinehttps://joom.ag/ey4QThis edition includes:Steem Interview with\xa0Ned ScottBitcoin News by\xa0Bitscan.comWAVES\xa0NewsICOcountdown.com report: DAOwn the Rabbit HoleThe Emerging Blockchain Network by\xa0Audo KryptowitzGUI Update"It is not simple, as we are trying to to build a good product, though we are on a good move. Now we in 0.1 development, in which we are covering Basic Wallet features:\xa0Account Creation/loginDashboard (Bitbox)Sending coin A &gt; coin ATransaction historyReference curr

In [21]:
# clean the data
clean_headlines = []

for i in range(0, len(headlines)):
    clean_headlines.append(re.sub("b[(')]", '', headlines[i])) #remove b'
    clean_headlines[i] = re.sub('b[(")]', '', clean_headlines[i]) #remove b"
    clean_headlines[i] = re.sub("\'", '', clean_headlines[i]) # remove \''
    

In [22]:
clean_headlines[0]

'SuperNET Weekly No. 18 CORE and CORE Media were birthed out of the community which formed around the SuperNET project launched by jl777 in 2014.\xa0Now that many aspects of the project are finally coming together in fantastic form and usable products are just around the bend, we couldnt be more pleased to help spread the word and keep our audience up to date on the latest happenings with this revolutionary project and its technology. Welcome to SuperNET Weekly!SuperNET Featured in CORE Magazinehttps://joom.ag/ey4QThis edition includes:Steem Interview with\xa0Ned ScottBitcoin News by\xa0Bitscan.comWAVES\xa0NewsICOcountdown.com report: DAOwn the Rabbit HoleThe Emerging Blockchain Network by\xa0Audo KryptowitzGUI Update"It is not simple, as we are trying to to build a good product, though we are on a good move. Now we in 0.1 development, in which we are covering Basic Wallet features:\xa0Account Creation/loginDashboard (Bitbox)Sending coin A &gt; coin ATransaction historyReference curren

In [23]:
# add clean headlines
Final_News_df["Combined_News"] = clean_headlines

Final_News_df["Combined_News"][0]

'SuperNET Weekly No. 18 CORE and CORE Media were birthed out of the community which formed around the SuperNET project launched by jl777 in 2014.\xa0Now that many aspects of the project are finally coming together in fantastic form and usable products are just around the bend, we couldnt be more pleased to help spread the word and keep our audience up to date on the latest happenings with this revolutionary project and its technology. Welcome to SuperNET Weekly!SuperNET Featured in CORE Magazinehttps://joom.ag/ey4QThis edition includes:Steem Interview with\xa0Ned ScottBitcoin News by\xa0Bitscan.comWAVES\xa0NewsICOcountdown.com report: DAOwn the Rabbit HoleThe Emerging Blockchain Network by\xa0Audo KryptowitzGUI Update"It is not simple, as we are trying to to build a good product, though we are on a good move. Now we in 0.1 development, in which we are covering Basic Wallet features:\xa0Account Creation/loginDashboard (Bitbox)Sending coin A &gt; coin ATransaction historyReference curren

In [24]:
Final_News_df

,title,body,categories,source_info.name,Coin_Symbol,Rev_time,Dt_Rev_time,Combined_News
0,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...
1,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...
2,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...
3,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...
4,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,DOGE,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...
...,...,...,...,...,...,...,...,...
344053,"Fintechs are hungry for DeFi yields, says Comp...","DeFi and traditional finance are converging, s...",ETH|Blockchain,TheBlock,DOGE,2021-07-06 19:05:16,2021-07-06,"Fintechs are hungry for DeFi yields, says Comp..."
344054,Why this is crucial to XRP’s trajectory over t...,The global cryptocurrency market noted a bit o...,XRP|Market|Trading|Altcoin,AMB Crypto,DOGE,2021-07-06 19:00:37,2021-07-06,Why this is crucial to XRP’s trajectory over t...
344055,Ripple CTO slams regulators as Barclays blocks...,Barclays UK customers have been blocked from t...,Exchange|XRP|Regulation|Fiat,CryptoSlate,DOGE,2021-07-06 19:00:21,2021-07-06,Ripple CTO slams regulators as Barclays blocks...
344056,Crypto Head: The U.S. Is the “Most-Ready” Nati...,According to a new report issued by cryptocurr...,BTC,Live Bitcoin News,DOGE,2021-07-06 19:00:11,2021-07-06,Crypto Head: The U.S. Is the “Most-Ready” Nati...


In [25]:
# get subjectivity & polarity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# create a function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [26]:
# Create columns
Final_News_df["Subjectivity"] = Final_News_df["Combined_News"].apply(getSubjectivity)
Final_News_df["Polarity"] = Final_News_df["Combined_News"].apply(getPolarity)

In [27]:
Final_News_df

,title,body,categories,source_info.name,Coin_Symbol,Rev_time,Dt_Rev_time,Combined_News,Subjectivity,Polarity
0,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579
1,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579
2,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579
3,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579
4,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,DOGE,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579
...,...,...,...,...,...,...,...,...,...,...
344053,"Fintechs are hungry for DeFi yields, says Comp...","DeFi and traditional finance are converging, s...",ETH|Blockchain,TheBlock,DOGE,2021-07-06 19:05:16,2021-07-06,"Fintechs are hungry for DeFi yields, says Comp...",0.541667,0.125000
344054,Why this is crucial to XRP’s trajectory over t...,The global cryptocurrency market noted a bit o...,XRP|Market|Trading|Altcoin,AMB Crypto,DOGE,2021-07-06 19:00:37,2021-07-06,Why this is crucial to XRP’s trajectory over t...,0.633333,0.108333
344055,Ripple CTO slams regulators as Barclays blocks...,Barclays UK customers have been blocked from t...,Exchange|XRP|Regulation|Fiat,CryptoSlate,DOGE,2021-07-06 19:00:21,2021-07-06,Ripple CTO slams regulators as Barclays blocks...,0.333333,0.250000
344056,Crypto Head: The U.S. Is the “Most-Ready” Nati...,According to a new report issued by cryptocurr...,BTC,Live Bitcoin News,DOGE,2021-07-06 19:00:11,2021-07-06,Crypto Head: The U.S. Is the “Most-Ready” Nati...,0.333788,0.137273


In [28]:
# get sentiment scores
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [29]:
# get scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(Final_News_df['Combined_News'])):
    SIA = getSIA(Final_News_df['Combined_News'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])

In [31]:
#store the sentiment
Final_News_df['Compound'] = compound
Final_News_df['Negative'] = neg
Final_News_df['Positive'] = pos
Final_News_df['Neutral'] = neu

In [32]:
Final_News_df

,title,body,categories,source_info.name,Coin_Symbol,Rev_time,Dt_Rev_time,Combined_News,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
0,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579,0.6277,0.020,0.050,0.930
1,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,XLM,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579,0.6277,0.020,0.050,0.930
2,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579,0.6277,0.020,0.050,0.930
3,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,BUSD,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579,0.6277,0.020,0.050,0.930
4,SuperNET Weekly No. 18,CORE and CORE Media were birthed out of the co...,Blockchain|Technology|Sponsored,NaN,DOGE,2016-07-11 00:00:00,2016-07-11,SuperNET Weekly No. 18 CORE and CORE Media wer...,0.572744,0.331579,0.6277,0.020,0.050,0.930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344053,"Fintechs are hungry for DeFi yields, says Comp...","DeFi and traditional finance are converging, s...",ETH|Blockchain,TheBlock,DOGE,2021-07-06 19:05:16,2021-07-06,"Fintechs are hungry for DeFi yields, says Comp...",0.541667,0.125000,-0.4404,0.075,0.000,0.925
344054,Why this is crucial to XRP’s trajectory over t...,The global cryptocurrency market noted a bit o...,XRP|Market|Trading|Altcoin,AMB Crypto,DOGE,2021-07-06 19:00:37,2021-07-06,Why this is crucial to XRP’s trajectory over t...,0.633333,0.108333,-0.7469,0.114,0.028,0.858
344055,Ripple CTO slams regulators as Barclays blocks...,Barclays UK customers have been blocked from t...,Exchange|XRP|Regulation|Fiat,CryptoSlate,DOGE,2021-07-06 19:00:21,2021-07-06,Ripple CTO slams regulators as Barclays blocks...,0.333333,0.250000,-0.3182,0.112,0.050,0.838
344056,Crypto Head: The U.S. Is the “Most-Ready” Nati...,According to a new report issued by cryptocurr...,BTC,Live Bitcoin News,DOGE,2021-07-06 19:00:11,2021-07-06,Crypto Head: The U.S. Is the “Most-Ready” Nati...,0.333788,0.137273,0.4767,0.000,0.045,0.955


In [ ]:
# Final news df take mean of all values grouping datewise


In [ ]:
# selecting rows based on condition
BTC_df = Final_News_df[(Final_News_df['Coin_Symbol'] == 'BTC')]
BTC_df

In [30]:
# import bitcoin 5 year data
Bitcoin_df = pd.read_csv("C:/UC_Berkeley/Analysis_Projects_Class_Folder/Module20_Final_Project/Cryptocurrency-Analysis/Data/bitcoin(fiveyears).csv")
Bitcoin_df

,Unnamed: 0,Name,Date,High,Low,Open,Close,Volume,Marketcap
0,1164,Bitcoin,2016-07-06 23:59:59,681.898010,670.418030,670.418030,677.330994,1.349610e+08,1.066146e+10
1,1165,Bitcoin,2016-07-07 23:59:59,682.432007,611.833984,678.090027,640.562012,2.580910e+08,1.008488e+10
2,1166,Bitcoin,2016-07-08 23:59:59,666.706970,636.466980,640.687988,666.523010,1.419700e+08,1.049604e+10
3,1167,Bitcoin,2016-07-09 23:59:59,666.383972,633.398987,666.383972,650.960022,1.805360e+08,1.025298e+10
4,1168,Bitcoin,2016-07-10 23:59:59,652.294006,641.263977,650.598999,649.359985,1.025320e+08,1.022900e+10
...,...,...,...,...,...,...,...,...,...
1821,2985,Bitcoin,2021-07-01 23:59:59,35035.982712,32883.781226,35035.982712,33572.117653,3.783896e+10,6.293393e+11
1822,2986,Bitcoin,2021-07-02 23:59:59,33939.588699,32770.680780,33549.600177,33897.048590,3.872897e+10,6.354508e+11
1823,2987,Bitcoin,2021-07-03 23:59:59,34909.259899,33402.696536,33854.421362,34668.548402,2.438396e+10,6.499397e+11
1824,2988,Bitcoin,2021-07-04 23:59:59,35937.567147,34396.477458,34665.564866,35287.779766,2.492431e+10,6.615748e+11


In [ ]:
Bitcoin_df['Label'] = np.where(Bitcoin_df['Close']Bitcoin_dfitcoin_df['Open']), 1, 0)
Bitcoin_df

In [ ]:
Bitcoin_df['Date'] = pd.to_datetime(Bitcoin_df.Date, format='%Y-%m-%d %H:%M:%S')
Bitcoin_df['Dt_Rev_time] = Bitcoin_df['Date'].dt.strftime('%Y-%m-%d')
Bitcoin_df.drop('Unnamed: 0', axis=1)
Bitcoin_NN_df= Bitcoin_df.drop(['Name','Date'], axis=1)
Bitcoin_NN_df.drop('Unnamed: 0', axis=1)
NN_df = pd.merge(Bitcoin_NN_df, BTC_df, how="left", on="Dt_Rev_time")
NN_df

In [ ]:
Final_NN_df = NN_df['Open', 'High', 'Low', 'Volume', 'Marketcap', 'Subjectivity', 'Compound', 'Negative', 'Positive', 'Label']
Final_NN_df